### 类

* 对象的or类的 数据成员


In [3]:
class Car(object):
    price=10000
    def __init__(self,color):
        self.color=color
        
    def get_price():
        return price
        
car_1=Car('red')
car_2=Car('blue')
print(Car.price)
Car.price=20000
print(car_1.price)
print(car_2.price)
print(Car.price)


10000
20000
20000
20000
